<a href="https://colab.research.google.com/github/reformship/kaggle/blob/master/attrition_predict0119.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd

train_file = r'train.csv';
test_file = r'test.csv';
train_employee_data = pd.read_csv(train_file, index_col='user_id');
test_employee_data = pd.read_csv(test_file, index_col='user_id');
print(train_employee_data.head());

from sklearn.model_selection import train_test_split
y = train_employee_data['Attrition'];
X = train_employee_data.drop(['Attrition'], axis=1);

#training set and validation set
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.99, test_size=0.01, random_state=0);

#numerical columns and categorical columns
numerical_cols = [col for col in X_train.columns if X_train[col].dtype in ['int64', 'float64']];
categorical_cols = [col for col in X_train.columns if X_train[col].dtype == 'object'];

#count the unique items in each categorical columns
print(train_employee_data[categorical_cols].nunique());

from sklearn.preprocessing import OneHotEncoder
#Use one-hot encoder to encode the categorical columns
OH_encoder = OneHotEncoder(sparse=False);
OH_train_cols = pd.DataFrame(OH_encoder.fit_transform(X_train[categorical_cols]));
OH_valid_cols = pd.DataFrame(OH_encoder.transform(X_val[categorical_cols]));
OH_test_cols = pd.DataFrame(OH_encoder.transform(test_employee_data[categorical_cols]));

OH_train_cols.index = X_train.index;
OH_valid_cols.index = X_val.index;
OH_test_cols.index = test_employee_data.index;

numerical_train_cols = X_train.drop(categorical_cols, axis=1);
numerical_valid_cols = X_val.drop(categorical_cols, axis=1);
numerical_test_cols = test_employee_data.drop(categorical_cols, axis=1);

OH_train = pd.concat([OH_train_cols, numerical_train_cols], axis=1);
OH_valid = pd.concat([OH_valid_cols, numerical_valid_cols], axis=1);
OH_test = pd.concat([OH_test_cols, numerical_test_cols], axis=1);

from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder();
y_train = label_encoder.fit_transform(y_train);
print(y_val);
y_val = label_encoder.transform(y_val);
print(y_val);

In [0]:
pip install tpot

In [0]:
from tpot import TPOTClassifier

In [0]:
tpot = TPOTClassifier()

In [0]:
tpot.fit(OH_train, y_train)
test_preds = tpot.predict_proba(OH_test)[:,1];
print(test_preds);
print(test_employee_data.index);
output = pd.DataFrame({'user_id': test_employee_data.index, 'Attrition': test_preds});
output.to_csv('submission5.csv', index=False);
print(output)
tpot.export('tpot_mnist_pipeline.py')